<a href="https://colab.research.google.com/github/injetiharsha/Crop-Disease-Prediction-/blob/main/Crop_Detection_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Step 1: Restart Runtime
#import os
#os._exit(0)  # This will force restart the runtime


In [3]:
# Step 2: Reinstall the correct versions
!pip install --upgrade sympy torchvision torch

# Step 3: Import everything again (after installation)
import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

# Step 4: Check if everything works
print("✅ Torch and torchvision loaded successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink-c

In [4]:
import os

dataset_zip_path = "/content/Plant_leave_diseases_dataset_without_augmentation.zip"

# Check file type
if os.path.exists(dataset_zip_path):
    print("File exists! Checking type...")
    !file /content/dataset.zip
else:
    print("File does not exist!")


File exists! Checking type...
/content/dataset.zip: cannot open `/content/dataset.zip' (No such file or directory)


### Import Dataset

In [6]:
import os
import zipfile
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define dataset zip file path
dataset_zip_path = "/content/Plant_leave_diseases_dataset_without_augmentation.zip"

# Extract dataset if not already extracted
dataset_root = "/content/Plant_leave_diseases_dataset_without_augmentation"  # Check actual folder name

if not os.path.exists(dataset_root):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")
else:
    print("Dataset already extracted. Skipping extraction.")

# Verify extracted files
print("Extracted files:", os.listdir("/content/"))

# Define the correct dataset path
dataset_path = dataset_root  # Adjust if images are inside a subfolder

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset folder not found at {dataset_path}. Check extracted files.")

# Define Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor()           # Convert to tensor
])

# Load Dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Verify Dataset Loaded
print("✅ Dataset loaded successfully!")
print(f"Total Images: {len(dataset)}")
print(f"Classes: {dataset.classes}")


Extracting dataset...
Extracted files: ['.config', 'Plant_leave_diseases_dataset_without_augmentation.zip', 'Plant_leave_diseases_dataset_without_augmentation', 'sample_data']
✅ Dataset loaded successfully!
Total Images: 16223
Classes: ['Background_without_leaves', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


### Split into Train and Test

In [7]:
indices = list(range(len(dataset)))

In [8]:
import numpy as np

split = int(np.floor(0.80 * len(dataset)))  # train_size

In [9]:
validation = int(np.floor(0.70 * split))  # validation

In [10]:
print(0, validation, split, len(dataset))

0 9084 12978 16223


In [11]:
print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :9084
length of validation size :3894
length of test size :7139


In [12]:
np.random.shuffle(indices)

In [13]:
train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

In [14]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [15]:
targets_size = len(dataset.class_to_idx)

### Original Modeling

In [16]:
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            # conv2
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            # conv3
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            # conv4
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(50176, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 9),
        )

    def forward(self, X):
        out = self.conv_layers(X)

        # Flatten
        out = out.view(-1, 50176)

        # Fully connected
        out = self.dense_layers(out)

        return out

In [17]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [23]:
device = "cpu"

In [24]:
model = CNN(targets_size)

In [25]:
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [26]:
!pip install torchsummary


from torchsummary import summary

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
       BatchNorm2d-3         [-1, 32, 224, 224]              64
            Conv2d-4         [-1, 32, 224, 224]           9,248
              ReLU-5         [-1, 32, 224, 224]               0
       BatchNorm2d-6         [-1, 32, 224, 224]              64
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,496
              ReLU-9         [-1, 64, 112, 112]               0
      BatchNorm2d-10         [-1, 64, 112, 112]             128
           Conv2d-11         [-1, 64, 112, 112]          36,928
             ReLU-12         [-1, 64, 112, 112]               0
      BatchNorm2d-13         [-1, 64, 112, 112]             128
        MaxPool2d-14           [-1, 64,

In [27]:
criterion = nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
optimizer = torch.optim.Adam(model.parameters())

In [28]:
device = "cpu"

In [29]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler,pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

# Batch Gradient Descent

In [30]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def calculate_f1(model, data_loader, device):
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)  # Get predicted class

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return f1_score(all_labels, all_preds, average="weighted")  # 'weighted' handles class imbalance



# Train

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score
import pickle  # ✅ Added for saving/loading results
from google.colab import files  # ✅ Added for auto-download

def calculate_accuracy(model, data_loader, device):
    """Computes accuracy of the model on a dataset."""
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
    return correct / total

def calculate_f1(model, data_loader, device):
    """Computes F1-score for model evaluation."""
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())
    return f1_score(all_labels, all_preds, average='weighted')

def batch_gd(model, criterion, train_loader, validation_loader, optimizer, device, epochs=20):
    """Trains a model using batch gradient descent and returns stored training results."""
    train_losses, validation_losses = [], []
    train_accuracies, validation_accuracies = [], []
    f1_scores = []

    for e in range(epochs):
        t0 = datetime.now()
        train_loss, train_correct, train_total = [], 0, 0

        # Training loop with progress bar
        model.train()
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {e+1}/{epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            train_loss.append(loss.item())

            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == targets).sum().item()
            train_total += targets.size(0)

        train_losses.append(np.mean(train_loss))  # Store train loss
        train_accuracies.append(train_correct / train_total)  # Store train accuracy

        # Validation loop
        model.eval()
        validation_loss, val_correct, val_total = [], 0, 0
        with torch.no_grad():
            for inputs, targets in validation_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                validation_loss.append(loss.item())

                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == targets).sum().item()
                val_total += targets.size(0)

        validation_losses.append(np.mean(validation_loss))  # Store val loss
        validation_accuracies.append(val_correct / val_total)  # Store val accuracy
        f1_scores.append(calculate_f1(model, validation_loader, device))  # Store F1-score

        print(f"Epoch {e+1}/{epochs} | Train Loss: {train_losses[-1]:.4f} | Val Loss: {validation_losses[-1]:.4f} | "
              f"Train Acc: {train_accuracies[-1]*100:.2f}% | Val Acc: {validation_accuracies[-1]*100:.2f}% | "
              f"F1-Score: {f1_scores[-1]:.4f} | Duration: {datetime.now() - t0}")

    return train_losses, validation_losses, train_accuracies, validation_accuracies, f1_scores

def plot_metrics(train_losses, val_losses, train_acc, val_acc, f1_scores):
    """Plots training loss, accuracy, and F1-score and saves it."""
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(15, 5))

    # Loss Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, train_losses, 'r', label='Train Loss')
    plt.plot(epochs, val_losses, 'b', label='Val Loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training & Validation Loss")
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, train_acc, 'r', label='Train Accuracy')
    plt.plot(epochs, val_acc, 'b', label='Val Accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Training & Validation Accuracy")
    plt.legend()

    # F1-Score Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, f1_scores, 'g', label='F1 Score')
    plt.xlabel("Epochs")
    plt.ylabel("F1 Score")
    plt.title("F1-Score over Epochs")
    plt.legend()

    plt.tight_layout()
    plt.savefig("training_plot.png")  # ✅ Auto-save the plot
    plt.show()

    # ✅ Auto-download in Google Colab
    files.download("training_plot.png")

# ✅ Training Only Once and Saving Results
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train only if results are not already saved
try:
    with open("training_results.pkl", "rb") as f:
        train_losses, val_losses, train_acc, val_acc, f1_scores = pickle.load(f)
    print("✅ Loaded previous training results. No retraining needed.")
except FileNotFoundError:
    print("🚀 Training model for the first time...")
    train_results = batch_gd(model, criterion, train_loader, validation_loader, optimizer, device, epochs=20)

    # Save results to avoid retraining
    with open("training_results.pkl", "wb") as f:
        pickle.dump(train_results, f)

    train_losses, val_losses, train_acc, val_acc, f1_scores = train_results

# ✅ Plot Metrics Separately (No Retraining)
plot_metrics(train_losses, val_losses, train_acc, val_acc, f1_scores)


🚀 Training model for the first time...


Epoch 1/20:   4%|▍         | 11/284 [03:41<1:26:01, 18.91s/it]

## Print Values

In [ ]:
print("Train Losses:", train_losses)
print("Validation Losses:", val_losses)
print("Train Accuracies:", train_acc)
print("Validation Accuracies:", val_acc)
print("F1 Scores:", f1_scores)


In [ ]:
def summarize_metrics(train_losses, val_losses, train_acc, val_acc, f1_scores):
    # 🔸 Averages
    print("AVERAGE VALUES ACROSS EPOCHS")
    print(f"Average Train Loss: {np.mean(train_losses):.4f}")
    print(f"Average Val Loss:   {np.mean(val_losses):.4f}")
    print(f"Average Train Acc:  {np.mean(train_acc) * 100:.2f}%")
    print(f"Average Val Acc:    {np.mean(val_acc) * 100:.2f}%")
    print(f"Average F1 Score:   {np.mean(f1_scores):.4f}\n")

    # 🔹 Best (Max accuracy / F1, Min loss)
    best_val_acc_idx = np.argmax(val_acc)
    best_f1_idx = np.argmax(f1_scores)
    lowest_val_loss_idx = np.argmin(val_losses)

    print("BEST EPOCHS")
    print(f"Best Val Accuracy:  {val_acc[best_val_acc_idx] * 100:.2f}% (Epoch {best_val_acc_idx + 1})")
    print(f"Best F1 Score:      {f1_scores[best_f1_idx]:.4f} (Epoch {best_f1_idx + 1})")
    print(f"Lowest Val Loss:    {val_losses[lowest_val_loss_idx]:.4f} (Epoch {lowest_val_loss_idx + 1})\n")

    # 🔹 Last Epoch
    print("LAST EPOCH")
    print(f"Last Train Loss:    {train_losses[-1]:.4f}")
    print(f"Last Val Loss:      {val_losses[-1]:.4f}")
    print(f"Last Train Acc:     {train_acc[-1] * 100:.2f}%")
    print(f"Last Val Acc:       {val_acc[-1] * 100:.2f}%")
    print(f"Last F1 Score:      {f1_scores[-1]:.4f}")

# ✅ Call this function after training
summarize_metrics(train_losses, val_losses, train_acc, val_acc, f1_scores)


# Training Metrics

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

def summarize_and_export_metrics(
    train_losses, val_losses, train_acc, val_acc, f1_scores,
    filename="detailed_training_summary.csv"
):
    epochs = list(range(1, len(train_losses) + 1))

    # Build epoch-wise dataframe
    df = pd.DataFrame({
        "Epoch": epochs,
        "Train Loss": train_losses,
        "Val Loss": val_losses,
        "Train Acc": [acc * 100 for acc in train_acc],
        "Val Acc": [acc * 100 for acc in val_acc],
        "F1 Score": f1_scores,
    })

    # Add average row
    avg_row = {
        "Epoch": "Average",
        "Train Loss": np.mean(train_losses),
        "Val Loss": np.mean(val_losses),
        "Train Acc": np.mean(train_acc) * 100,
        "Val Acc": np.mean(val_acc) * 100,
        "F1 Score": np.mean(f1_scores),
    }

    # Add best metrics
    best_val_acc_idx = np.argmax(val_acc)
    best_f1_idx = np.argmax(f1_scores)
    lowest_val_loss_idx = np.argmin(val_losses)

    best_rows = [
        {
            "Epoch": f"Best Val Acc (Epoch {best_val_acc_idx+1})",
            "Train Loss": train_losses[best_val_acc_idx],
            "Val Loss": val_losses[best_val_acc_idx],
            "Train Acc": train_acc[best_val_acc_idx] * 100,
            "Val Acc": val_acc[best_val_acc_idx] * 100,
            "F1 Score": f1_scores[best_val_acc_idx],
        },
        {
            "Epoch": f"Best F1 Score (Epoch {best_f1_idx+1})",
            "Train Loss": train_losses[best_f1_idx],
            "Val Loss": val_losses[best_f1_idx],
            "Train Acc": train_acc[best_f1_idx] * 100,
            "Val Acc": val_acc[best_f1_idx] * 100,
            "F1 Score": f1_scores[best_f1_idx],
        },
        {
            "Epoch": f"Lowest Val Loss (Epoch {lowest_val_loss_idx+1})",
            "Train Loss": train_losses[lowest_val_loss_idx],
            "Val Loss": val_losses[lowest_val_loss_idx],
            "Train Acc": train_acc[lowest_val_loss_idx] * 100,
            "Val Acc": val_acc[lowest_val_loss_idx] * 100,
            "F1 Score": f1_scores[lowest_val_loss_idx],
        },
    ]

    # Add last epoch
    last_row = {
        "Epoch": "Last Epoch",
        "Train Loss": train_losses[-1],
        "Val Loss": val_losses[-1],
        "Train Acc": train_acc[-1] * 100,
        "Val Acc": val_acc[-1] * 100,
        "F1 Score": f1_scores[-1],
    }

    # Append everything to the final DataFrame
    summary_df = pd.concat(
        [df, pd.DataFrame([avg_row] + best_rows + [last_row])],
        ignore_index=True
    )

    # Export
    summary_df.to_csv(filename, index=False)
    print(f"✅ Full metrics exported to: {filename}")
    files.download(filename)


In [ ]:
summarize_and_export_metrics(train_losses, val_losses, train_acc, val_acc, f1_scores)


## Precision


In [ ]:
from sklearn.metrics import precision_score, recall_score

def calculate_precision_recall(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    return precision, recall

# ✅ Example usage after training
precision, recall = calculate_precision_recall(model, validation_loader, device)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")


## Confusion Matrix


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from google.colab import files
import matplotlib.pyplot as plt
import torch

def plot_confusion(model, dataloader, class_names, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    # ✅ Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    # ✅ Plot
    fig, ax = plt.subplots(figsize=(20, 20))
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.tight_layout()

    # ✅ Save and Download
    fig.savefig("confusion_matrix.png")
    plt.show()

    # ✅ Download (Only works in Colab!)
    files.download("confusion_matrix.png")


### Plot Confusion Matrix


In [ ]:
class_names = [
    'Background_without_leaves',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

plot_confusion(model, validation_loader, class_names, device)


# Save model


In [ ]:
torch.save(model.state_dict() , 'CDPmodel5.1.pth')

from google.colab import files

files.download("CDPmodel5.1.pth")  # Download the model to your local machine


### Load Model

In [ ]:
targets_size = 9
model = CNN(targets_size)
model.load_state_dict(torch.load("CDPmodel5.1.pth"))  # Load weights
model.to(device)  # Move to GPU if needed


In [ ]:
def evaluate_on_test_data(model, test_loader, device, class_names):
    from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    print("\n📊 Classification Report on Test Data:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap="Blues", xticks_rotation=180)
    plt.tight_layout()
    plt.savefig("test_confusion_matrix.png")
    plt.show()
    files.download("test_confusion_matrix.png")


In [ ]:
evaluate_on_test_data(model, test_loader, device, class_names=[
    'Background_without_leaves',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
])
